# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [44]:
import json

from bnbprob.pfssp import CallbackBnB, plot_gantt
# from bnbprob.pfssp.problem import PermFlowShopLazy
from bnbprob.pfssp.cython.problem import PermFlowShopLazy
# from bnbprob.pfssp.cython.search import configure_logfile
from bnbpy.cython.search import configure_logfile
# from bnbpy import configure_logfile

In [45]:
configure_logfile("pfssp-bench.log", mode="w")

In [51]:
with open("./../data/flow-shop/reC11.json", mode="r", encoding="utf8") as f:
    p = json.load(f)

print(f"{len(p)} Jobs; {len(p[0])} Machines")

20 Jobs; 10 Machines


In [52]:
import time


class MyFSSP(PermFlowShopLazy):
    branch_time = 0
    lb1_time = 0
    lb2_time = 0
    ls_time = 0

    # def warmstart(self):
    #     pass

    def branch(self):
        s = time.time()
        out = super().branch()
        type(self).branch_time += (time.time() - s)
        return out

    def local_search(self):
        s = time.time()
        out = super().local_search()
        type(self).ls_time += (time.time() - s)
        return out

    def calc_bound(self):
        s = time.time()
        out = super().calc_bound()
        type(self).lb1_time += (time.time() - s)
        return out

    def bound_upgrade(self):
        s = time.time()
        out = super().bound_upgrade()
        type(self).lb2_time += (time.time() - s)
        return out


class MyBnB(CallbackBnB):
    branch_time = 0

    def branch(self, node):
        s = time.time()
        out = super().branch(node)
        type(self).branch_time += (time.time() - s)
        return out

In [56]:
problem_sm = MyFSSP.from_p(p, constructive='neh')
bnb_sm = MyBnB(
    eval_node='in', rtol=0.0001, restart_freq=200, save_tree=False
)

In [57]:
sol_sm = bnb_sm.solve(
    problem_sm, maxiter=1000000, timelimit=600
)
print(sol_sm)

Status: OPTIMAL | Cost: 1431 | LB: 1431


In [55]:
print(MyBnB.branch_time)
print(MyFSSP.branch_time)
print(MyFSSP.ls_time)
print(MyFSSP.lb1_time)
print(MyFSSP.lb2_time)

2.615528106689453
0.6043441295623779
0.0586094856262207
0.13035035133361816
1.2085516452789307


30.078226804733276
11.00402045249939
1.0040879249572754
6.359635829925537
11.666419267654419

In [ ]:
plot_gantt(sol_sm.sequence, dpi=120, seed=42, figsize=[8, 3])

In [ ]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))
model.K = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.K, within=pyo.Binary)
model.h = pyo.Var(model.M, model.K, within=pyo.NonNegativeReals)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_position(model, k):
    return sum(model.x[j, k] for j in model.J) == 1


def cstr_job(model, j):
    return sum(model.x[j, k] for k in model.K) == 1


def cstr_seq(model, m, k):
    if k == model.K.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m, k + 1]
    )


def cstr_precede(model, m, k):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m + 1, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, m):
    k = model.K.last()
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.C
    )


model.cstr_position = pyo.Constraint(model.K, rule=cstr_position)
model.cstr_job = pyo.Constraint(model.K, rule=cstr_job)
model.cstr_seq = pyo.Constraint(model.M, model.K, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(model.M, model.K, rule=cstr_precede)
model.cstr_total_time = pyo.Constraint(model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
solver = pyo.SolverFactory('appsi_highs')
solver.options['mip_heuristic_effort'] = 0.1
solver.options['time_limit'] = 600
solver.options['log_file'] = 'Highs.log'
solver.solve(model, tee=True)

In [ ]:
# import math

# math.log10(math.factorial(20))

In this implmentation lower bounds are computed by
the max of a single machine and
a two machine relaxations.

The bounds for single and two-machine problems are described
by Potts (1980), also implemented by Ladhari & Haouari (2005),
therein described as 'LB1' and 'LB5'.

The warmstart strategy is proposed by Palmer (1965).

## References

Ladhari, T., & Haouari, M. (2005). A computational study of
the permutation flow shop problem based on a tight lower bound.
Computers & Operations Research, 32(7), 1831-1847.

Potts, C. N. (1980). An adaptive branching rule for the permutation
flow-shop problem. European Journal of Operational Research, 5(1), 19-25.

Palmer, D. S. (1965). Sequencing jobs through a multi-stage process
in the minimum total time—a quick method of obtaining a near optimum.
Journal of the Operational Research Society, 16(1), 101-107

## Bonus - MILP Model

This is the usual Disjunctive MILP model as an alternative to compare performance.

### Position-based model

```python
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))
model.K = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.K, within=pyo.Binary)
model.h = pyo.Var(model.M, model.K, within=pyo.NonNegativeReals)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_position(model, k):
    return sum(model.x[j, k] for j in model.J) == 1


def cstr_job(model, j):
    return sum(model.x[j, k] for k in model.K) == 1


def cstr_seq(model, m, k):
    if k == model.K.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m, k + 1]
    )


def cstr_precede(model, m, k):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.h[m + 1, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, m):
    k = model.K.last()
    return (
        model.h[m, k] + sum(model.p[j, m] * model.x[j, k] for j in model.J)
        <= model.C
    )


model.cstr_position = pyo.Constraint(model.K, rule=cstr_position)
model.cstr_job = pyo.Constraint(model.K, rule=cstr_job)
model.cstr_seq = pyo.Constraint(model.M, model.K, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(model.M, model.K, rule=cstr_precede)
model.cstr_total_time = pyo.Constraint(model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
solver = pyo.SolverFactory('appsi_highs')
solver.options['mip_heuristic_effort'] = 0.1
solver.options['time_limit'] = 120
solver.options['log_file'] = 'Highs.log'
solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 120
solver.solve(model, tee=True)
```


### Disjunctive model

In experiments with 15 jobs and 8 machines, the Lazy implementation solved in 24s whereas with a time limit of 120s Gurobi could not reach the optimal solution (with a MIP gap of nearly 25%).

```python
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets for machines, jobs, horizon, and job sequences
model.M = pyo.Set(initialize=range(len(p[0])))
model.J = pyo.Set(initialize=range(len(p)))

# Parameters
model.p = pyo.Param(model.J, model.M, initialize=lambda _, m, j: p[m][j])
model.V = pyo.Param(initialize=sum(pim for pi in p for pim in pi))

# Variables
model.x = pyo.Var(model.J, model.M, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.J, model.J, within=pyo.Binary)
model.C = pyo.Var(within=pyo.NonNegativeReals)


# Constraints
def cstr_seq(model, j, m):
    if m == model.M.last():
        return pyo.Constraint.Skip
    return model.x[j, m] + model.p[j, m] <= model.x[j, m + 1]


def cstr_precede(model, j, k, m):
    return model.x[j, m] + model.p[j, m] <= model.x[k, m] + model.V * (
        1 - model.z[j, k]
    )


def cstr_comp_precede(model, j, k):
    if j == k:
        return model.z[j, k] + model.z[k, j] == 0.0
    return model.z[j, k] + model.z[k, j] == 1.0


def cstr_total_time(model, j, m):
    return model.x[j, m] + model.p[j, m] <= model.C


model.cstr_seq = pyo.Constraint(model.J, model.M, rule=cstr_seq)
model.cstr_precede = pyo.Constraint(
    model.J, model.J, model.M, rule=cstr_precede
)
model.cstr_comp_precede = pyo.Constraint(
    model.J, model.J, rule=cstr_comp_precede
)
model.cstr_total_time = pyo.Constraint(model.J, model.M, rule=cstr_total_time)


# Objective
model.obj = pyo.Objective(expr=model.C, sense=pyo.minimize)

# HiGHS
solver = pyo.SolverFactory("appsi_highs")
solver.options["mip_heuristic_effort"] = 0.1
solver.options["time_limit"] = 120
solver.options["log_file"] = "Highs.log"
solver.solve(model, tee=True)

# Gurobi
solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["Heuristics"] = 0.2
solver.options["Cuts"] = 2
solver.options["TimeLimit"] = 120
solver.solve(model, tee=True)
```

## Classes to double-check computing times

```python
import time


class MyFSSP(PermFlowShopLazy):
    branch_time = 0
    lb1_time = 0
    lb2_time = 0
    ls_time = 0

    # def warmstart(self):
    #     pass

    def branch(self):
        s = time.time()
        out = super().branch()
        type(self).branch_time += (time.time() - s)
        return out

    def local_search(self):
        s = time.time()
        out = super().local_search()
        type(self).ls_time += (time.time() - s)
        return out

    def calc_bound(self):
        s = time.time()
        out = super().calc_bound()
        type(self).lb1_time += (time.time() - s)
        return out

    def bound_upgrade(self):
        s = time.time()
        out = super().bound_upgrade()
        type(self).lb2_time += (time.time() - s)
        return out


class MyBnB(CallbackBnB):
    branch_time = 0

    def branch(self, node):
        s = time.time()
        out = super().branch(node)
        type(self).branch_time += (time.time() - s)
        return out
```